In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import date
import re
import os

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")

service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.daraz.com.np/laptops/")

In [3]:
links = []
for i in range(0, 20):
    products = driver.find_elements(By.XPATH, '//*[@id="id-a-link"]')
    urls = [x.get_property("href") for x in products]
    links += urls
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1500);")
    next = driver.find_element(
        By.XPATH,
        '//*[@id="root"]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[9]/a',
    )
    next.click()
    time.sleep(1)

In [4]:
df = pd.DataFrame(links, columns=['Url'])
df.to_csv('../datasets/laptops_url.csv', index=False)
link = df["Url"]
# https://www.daraz.com.np/products/acer-nitro-5-an515-47-gaming-laptop-amd-ryzen-7-7735hs-octa-core-processor-16gb-ddr5-4800mhz-ram-512gb-pcle-nvme-ssd-nvidia-geforce-rtx-3050-4gb-graphics-156-165hz-fhd-display-obsidian-black-i127988100-s1035028144.html?search=1

In [5]:
for x in range(21,100):
    print(link[x])
    driver.get(link[x])
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[2]/i',)))
        driver.find_element(By.XPATH, '/html/body/div[2]/i')
        print("out of stock")   
    except:
        try:
            actual_price = driver.find_element(
                By.XPATH, '//*[@id="module_product_price_1"]/div/div/div/span[1]'
            ).text

            discount_price = driver.find_element(
                By.XPATH, '//*[@id="module_product_price_1"]/div/div/span'
            ).text
        except:
            actual_price = driver.find_element(
                By.XPATH, '//*[@id="module_product_price_1"]/div/div/span'
            ).text

            discount_price = actual_price

        today = date.today()
        df = pd.DataFrame([[today, actual_price, discount_price]], columns=["Date", "Actual Price", "Discount Price"])
        df["Actual Price"] = df["Actual Price"].str.replace("Rs. ", "").str.replace(",", "")
        df["Discount Price"] = df["Discount Price"].str.replace("Rs. ", "").str.replace(",", "")
        name = str(link[x])
        name = re.sub(r'[^a-zA-Z0-9]', '', name)
        name = re.sub(r'httpswwwdarazcomnpproducts', '', name)
        filename = "../datasets/prices/" + name + ".csv"
        df.to_csv(filename, mode='a', header = not os.path.exists(filename), index=False)
    time.sleep(1)


https://www.daraz.com.np/products/avita-magus-cosmos-2in1-ns12t5in025p-n4000-i108930357-s1029438113.html?search=1
https://www.daraz.com.np/products/lenovo-ideapad-slim-3-i3-13th-gen-8gb-ram-256gb-ssd-156-fhd-laptop-i128202161-s1035310879.html?search=1
https://www.daraz.com.np/products/acer-swift-3-ryzen7-7500u-8gb512gb14inch-displaybacklit-keyboard-i129437422-s1037427645.html?search=1
https://www.daraz.com.np/products/hp-victus-15-core-i5-13th-gen-intel-13420h-nvidia-rtx-3050-6gb-graphics-16gb-ram512gb-ssd156-144hrzbacklit-keyboardwin-11-i128174010-s1035273454.html?search=1
https://www.daraz.com.np/products/acer-aspire-5-core-i3-11th-gen-8gb-ram-128gb-ssd-1tb-hdd-156-fhd-win10-genuine-laptop-i113601038-s1030884979.html?search=1
https://www.daraz.com.np/products/acer-aspire-5-i5-13th-generation-8gb-ddr5-ram-512gb-nvme-ssd-intel-iris-xe-graphics-14-fhd-ips-display-steel-grey-i128212224-s1035333820.html?search=1
https://www.daraz.com.np/products/asus-zephyrus-g15-ga503qe-ryzen-9-5900hs-rt

In [6]:
# https://www.daraz.com.np/products/acer-nitro-5-an515-47-gaming-laptop-amd-ryzen-7-7735hs-octa-core-processor-16gb-ddr5-4800mhz-ram-512gb-pcle-nvme-ssd-nvidia-geforce-rtx-3050-4gb-graphics-156-165hz-fhd-display-obsidian-black-i127988100-s1035028144.html?search=1